In [3]:
import numpy as np

In [26]:
def compute_credit_score(oper_margin, tot_debt_to_tot_eqy, tot_debt_to_ebitda, ebitda_to_tot_int_exp, return_on_asset, asset_turnover):
    # Normalize and invert metrics where necessary; assumed to be pre-processed
    metrics = [
        oper_margin,                # Higher is better
        1 - tot_debt_to_tot_eqy,    # Invert because lower debt to equity is better
        1 - tot_debt_to_ebitda,     # Invert because lower debt to EBITDA is better
        ebitda_to_tot_int_exp,      # Higher is better
        return_on_asset,            # Higher is better
        asset_turnover              # Higher is better
    ]
    
    # Weights based on the above discussion
    weights = [0.20, 0.20, 0.20, 0.15, 0.15, 0.10]
    
    # Compute weighted score, adjust for scaling
    score = sum(m * w for m, w in zip(metrics, weights)) * 7.5 + 2.5
    
    # Determine credit rating
    ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]
    rating_index = min(int(score - 2.5), 8)
    
    return score, ratings[rating_index]

# Example usage
credit_score, credit_rating = compute_credit_score(
    oper_margin=0.8, 
    tot_debt_to_tot_eqy=0.4, 
    tot_debt_to_ebitda=0.3, 
    ebitda_to_tot_int_exp=0.7, 
    return_on_asset=0.5, 
    asset_turnover=0.6
)
print(f"Credit Score: {credit_score:.2f}, Credit Rating: {credit_rating}")


Credit Score: 7.45, Credit Rating: Ba


In [28]:
score = 2.5
ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]
rating_index = min(int(score - 2.5), 8)
ratings[rating_index]

'Aaa'

In [30]:
def compute_credit_score(oper_margin, tot_debt_to_tot_eqy, tot_debt_to_ebitda, ebitda_to_tot_int_exp, return_on_asset, asset_turnover):
    metrics = [oper_margin, 1 - tot_debt_to_tot_eqy, 1 - tot_debt_to_ebitda, ebitda_to_tot_int_exp, return_on_asset, asset_turnover]
    weights = [0.20, 0.20, 0.20, 0.15, 0.15, 0.10]  # Weights for each metric

    score = sum(m * w for m, w in zip(metrics, weights)) * 7.5 + 2.5
    score = min(max(score, 2.5), 8.5)

    ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]
    rating_index = max(int(score - 2.5), 0)  
    return score, ratings[rating_index]

# Example usage
credit_score, credit_rating = compute_credit_score(0.8, 0.4, 0.3, 0.7, 0.5, 0.6)
print(f"Credit Score: {credit_score:.2f}, Credit Rating: {credit_rating}")


Credit Score: 7.45, Credit Rating: Ba


In [32]:
credit_score, credit_rating = compute_credit_score(
    oper_margin=0.8, 
    tot_debt_to_tot_eqy=0.4, 
    tot_debt_to_ebitda=0.3, 
    ebitda_to_tot_int_exp=0.7, 
    return_on_asset=0.5, 
    asset_turnover=0.6
)

credit_score, credit_rating

(7.449999999999999, 'Ba')

In [33]:
def compute_credit_score(settings, **metrics):
    overall_score = sum(
        cat_details["weight"] * sum(
            (1 - metrics[metric] if detail["direction"] == "lower_better" else metrics[metric]) * detail["weight"]
            for metric, detail in cat_details["metrics"].items()
        )
        for category, cat_details in settings["categories"].items()
    )

    # Scale and bound the score within the rating scale
    overall_score = min(max(overall_score * 7.5 + 2.5, 2.5), 8.5)

    # Map score to rating
    ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]
    rating_index = max(int(overall_score - 2.5), 0)
    return overall_score, ratings[rating_index]

# Example settings
settings = {
    "categories": {
        "profitability": {"weight": 0.3, "metrics": {"oper_margin": {"weight": 1.0, "direction": "higher_better"}}},
        "leverage_coverage": {"weight": 0.4, "metrics": {
            "tot_debt_to_tot_eqy": {"weight": 0.35, "direction": "lower_better"},
            "tot_debt_to_ebitda": {"weight": 0.35, "direction": "lower_better"},
            "ebitda_to_tot_int_exp": {"weight": 0.30, "direction": "higher_better"}
        }},
        "efficiency": {"weight": 0.3, "metrics": {
            "return_on_asset": {"weight": 0.60, "direction": "higher_better"},
            "asset_turnover": {"weight": 0.40, "direction": "higher_better"}
        }}
    }
}

# Example usage
credit_score, credit_rating = compute_credit_score(
    settings,
    oper_margin=0.8,
    tot_debt_to_tot_eqy=0.4,
    tot_debt_to_ebitda=0.3,
    ebitda_to_tot_int_exp=0.7,
    return_on_asset=0.5,
    asset_turnover=0.6
)
print(f"Credit Score: {credit_score:.2f}, Credit Rating: {credit_rating}")


Credit Score: 7.51, Credit Rating: B


In [34]:
def determine_metric_rating(value, thresholds):
    """Determine the rating index based on the metric value and its thresholds."""
    for index, (low, high) in enumerate(thresholds):
        if low <= value <= high:
            return index
    return len(thresholds) - 1  # Default to the last rating if out of range

def compute_credit_score(settings, **metrics):
    ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]
    category_scores = []

    for category, cat_details in settings["categories"].items():
        metric_scores = []
        for metric, details in cat_details["metrics"].items():
            metric_value = metrics[metric]
            if details["lower_is_better"]:
                metric_value = -metric_value  # Invert metric value for lower_is_better metrics
            rating_index = determine_metric_rating(metric_value, details["thresholds"])
            metric_scores.append((9 - rating_index) * details["weight"])  # Convert index to score, higher is better
        
        # Aggregate metric scores within the category
        category_score = sum(metric_scores) / sum(details["weight"] for details in cat_details["metrics"].values())
        category_scores.append(category_score * cat_details["weight"])

    # Aggregate category scores to determine the final rating
    final_score = sum(category_scores)
    final_rating_index = min(max(int(final_score), 0), len(ratings) - 1)
    return ratings[final_rating_index]

# Example usage
settings = {
    "categories": {
        "profitability": {
            "weight": 0.3,
            "metrics": {
                "oper_margin": {
                    "weight": 1.0,
                    "lower_is_better": False,
                    "thresholds": [
                        (40, float("inf")),
                        (35, 39),
                        (30, 34),
                        (25, 29),
                        (20, 24),
                        (15, 19),
                        (10, 14),
                        (5, 9),
                        (float("-inf"), 4)
                    ]
                },
            }
        },
        # Example includes one category for brevity; others would follow similarly
    }
}

# Example metric input
credit_rating = compute_credit_score(
    settings,
    oper_margin=32  # Example value
)
print(f"Credit Rating: {credit_rating}")


Credit Rating: A


In [41]:
settings = {
    "categories": {
        "profitability": {
            "weight": 0.3,
            "metrics": {
                "oper_margin": {
                    "weight": 1.0,
                    "lower_is_better": False,
                    "thresholds": [
                        (25, float("inf")),
                        (20, 24.99),
                        (15, 19.99),
                        (10, 14.99),
                        (5, 9.99),
                        (0, 4.99),
                        (-5, -0.01),
                        (-10, -5.01),
                        (float("-inf"), -10.01),
                    ]
                }
            }
        },
        "leverage_coverage": {
            "weight": 0.4,
            "metrics": {
                "tot_debt_to_tot_eqy": {
                    "weight": 0.35,
                    "lower_is_better": True,
                    "thresholds": [
                        (float("-inf"), 0.5),
                        (0.51, 1),
                        (1.01, 2),
                        (2.01, 3),
                        (3.01, 4),
                        (4.01, 5),
                        (5.01, 6),
                        (6.01, float("inf")),
                    ]
                },
                "tot_debt_to_ebitda": {
                    "weight": 0.35,
                    "lower_is_better": True,
                    "thresholds": [
                        (float("-inf"), 2),
                        (2.01, 3.5),
                        (3.51, 5),
                        (5.01, 6.5),
                        (6.51, 8),
                        (8.01, 10),
                        (10.01, 12),
                        (12.01, float("inf")),
                    ]
                },
                "ebitda_to_tot_int_exp": {
                    "weight": 0.3,
                    "lower_is_better": False,
                    "thresholds": [
                        (10, float("inf")),
                        (7, 9.99),
                        (5, 6.99),
                        (3, 4.99),
                        (2, 2.99),
                        (1, 1.99),
                        (0.5, 0.99),
                        (0, 0.49),
                        (float("-inf"), 0),
                    ]
                }
            }
        },
        "efficiency": {
            "weight": 0.3,
            "metrics": {
                "return_on_asset": {
                    "weight": 0.6,
                    "lower_is_better": False,
                    "thresholds": [
                        (15, float("inf")),
                        (10, 14.99),
                        (7, 9.99),
                        (5, 6.99),
                        (3, 4.99),
                        (2, 2.99),
                        (1, 1.99),
                        (0, 0.99),
                        (float("-inf"), 0),
                    ]
                },
                "asset_turnover": {
                    "weight": 0.4,
                    "lower_is_better": False,
                    "thresholds": [
                        (2, float("inf")),
                        (1.5, 1.99),
                        (1, 1.49),
                        (0.8, 0.99),
                        (0.6, 0.79),
                        (0.4, 0.59),
                        (0.2, 0.39),
                        (0.1, 0.19),
                        (float("-inf"), 0.09),
                    ]
                }
            }
        }
    }
}


In [42]:
def get_rating_index_for_metric(value, thresholds):
    """
    Determine the rating index based on the metric's value against defined thresholds.
    Each threshold defines a range for a rating, from highest to lowest.
    """
    for index, (lower_bound, upper_bound) in enumerate(thresholds):
        if lower_bound <= value <= upper_bound:
            return index
    return len(thresholds) // 2 # else return the middle rating 

def calculate_category_score(category_details, metric_values):
    """
    Calculate the weighted average rating index for a category based on its metrics.
    """
    metric_scores_sum = 0
    total_weight = 0
    for metric_name, metric_details in category_details["metrics"].items():
        metric_value = metric_values[metric_name]
        
        if metric_details["lower_is_better"]:
            metric_value = -metric_value
        rating_index = get_rating_index_for_metric(metric_value, metric_details["thresholds"])
        metric_scores_sum += (9 - rating_index) * metric_details["weight"]  # 9 - rating_index to make higher better
        total_weight += metric_details["weight"]
    
    return metric_scores_sum / total_weight

def compute_final_credit_rating(settings, **metrics):
    """
    Compute the final credit rating based on metrics and settings. 
    """
    ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]
    total_score = 0
    
    for category_name, category_details in settings["categories"].items():
        category_score = calculate_category_score(category_details, metrics)
        total_score += category_score * category_details["weight"]
    
    # Convert total score to an index within the ratings list
    ratings = ["Aaa", "Aa", "A", "Baa", "Ba", "B", "Caa", "Ca", "C"]

    final_rating_index = min(max(int(total_score), 0), len(ratings) - 1)
    return ratings[final_rating_index]



# Compute the credit rating for an example set of metric values
credit_rating = compute_final_credit_rating(
    settings,
    oper_margin=0.8, 
    tot_debt_to_tot_eqy=0.4, 
    tot_debt_to_ebitda=0.3, 
    ebitda_to_tot_int_exp=0.7, 
    return_on_asset=0.5, 
    asset_turnover=0.6
)
print(f"Credit Rating: {credit_rating}")


Credit Rating: B
